In [1]:
!pip install tensorflow
!pip install tensorflow-text
#!pip install spacy
!pip install contractions
!pip install emoji
!pip install emot
!pip install demoji

ERROR: Could not find a version that satisfies the requirement tensorflow-text (from versions: none)
ERROR: No matching distribution found for tensorflow-text


In [2]:
#import spacy

import pandas as pd
import numpy as np
import string
import contractions
#from nltk.stem import WordNetLemmatizer
#from nltk.stem import PorterStemmer
import re
import regex
import demoji
import emoji
from sklearn.metrics import f1_score
import os
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('whitespace')
#nltk.download('wordnet')
#nltk.download('omw-1.4')
stop_words = set(stopwords.words("english"))

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading whitespace: Package 'whitespace' not found
[nltk_data]     in index


In [4]:
df_train = pd.read_csv('training_t1_sp.csv')
df_val = pd.read_csv('Task 1 - Validation set .csv')
#df_test=pd.read_csv('test - no_label.csv')

In [ ]:
# df_train = df_train.rename(columns={'text                ': 'Text'})   #, 'category': 'Category'})
# df_val = df_val.rename(columns={'text                        ': 'Text'})  #, 'category': 'Category'})
# df_test = df_test.rename(columns={'text                        ': 'Text'})   #, 'category': 'Category'})

In [5]:
df_train

,id,text,category
0,1409573191098511365,Si quieres saber lo diverso e inclusivo que es...,nhs
1,1413030472020631553,Y ya esta ya me he liberado.Es que las injusti...,hs
2,1409489246608560128,me molesta mucho el discurso ese en favor del ...,nhs
3,1412169304372568070,Yo te lo explico.No se pueden cortar las vías...,hs
4,1410109561814622210,"8. No somos opositores al movimiento #LGTBI+, ...",nhs
...,...,...,...
1395,1409474601831309317,Hoy es el día del Orgullo y tengo mil razones ...,hs
1396,1414347745427697667,"Y sí, señores. Muchas veces la gente lgtb sal...",nhs
1397,1409797952860721153,Un fuerte abrazo desde Andalucía y nuestras f...,hs
1398,1411689668433829890,Se siguen perdiendo oportunidades en materia ...,nhs


In [ ]:
# df_train = df_train.drop(['Unnamed: 0' ], axis=1)
# df_val = df_val.drop(['Unnamed: 0' ], axis=1)
# df_test = df_test.drop(['Unnamed: 0' ], axis=1)

In [6]:
df_train.category.value_counts()

nhs    700
hs     700
Name: category, dtype: int64

# Oversampling is required for Transphobia and Homophobia

In [7]:
english_stopwords = stopwords.words('english')
exclude = set(string.punctuation)

def emo(text):
  temp=emoji.demojize(text,delimiters=(" "," "))
  temp=temp.replace("_","  ")
  return temp

def preprocess(text):
  #text=demoji.findall(df['Text'])
  text = contractions.fix(text.lower(), slang=True)
  text =re.sub("@ ?[A-Za-z0-9_]+", "", text)
  text= re.sub(r'\d+', '', text)
  text=re.sub(r'$', '', text)
  text= re.sub(r'’','', text )
  text=re.sub('<.*?>','',text)
  text=re.sub(r'http\S+', '', text)
  #text=emoji.demojize(text, delimiters=(" ", " "))
  text = ''.join(ch for ch in text if ch not in exclude)
  tokens = word_tokenize(text)
  #print("Tokens:", tokens)
  text = [t for t in tokens if t not in english_stopwords]
  text = " ".join(text)
  return text

def text_preprocess(text):
  temp = []
  for line in sent_tokenize(text):
    temp.append(preprocess(line))
  return temp

In [9]:
df_train['clean_text']=df_train['text'].apply(lambda x:emo(x))
df_train['clean_text']=df_train['clean_text'].apply(lambda X: text_preprocess(X))

df_val['clean_text']=df_val['text'].apply(lambda x:emo(x))
df_val["clean_text"]=df_val['clean_text'].apply(lambda X: text_preprocess(X))

# df_test['clean_text']=df_test['text'].apply(lambda x:emo(x))
# df_test["clean_text"]=df_test['clean_text'].apply(lambda X: text_preprocess(X))

In [ ]:
from sklearn.utils import resample
#create two different dataframe of majority and minority class
df_majority = df_train[(df_train['multiclass']=='Not Hope')]
df_minority1 = df_train[(df_train['multiclass']=='Generalized Hope')]
df_minority2 = df_train[(df_train['multiclass']=='Realistic Hope')]
df_minority3 = df_train[(df_train['multiclass']=='Unrealistic Hope')]
# upsample minority class
df_minority_upsampled1 = resample(df_minority1,replace=True, n_samples= 3088, random_state=None)  # reproducible results
df_minority_upsampled2 = resample(df_minority2,replace=True, n_samples= 3088, random_state=None)
df_minority_upsampled3 = resample(df_minority3,replace=True, n_samples= 3088, random_state=None)
#df_minority_upsampled1 = resample(df_minority1,replace=True, n_samples= 3088, random_state=None)
# Combine majority class with upsampled minority class
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_minority_upsampled1,df_minority_upsampled2,df_minority_upsampled3,df_majority])

In [10]:
df_train['category'].value_counts()

nhs    700
hs     700
Name: category, dtype: int64

In [12]:
# Import label encoder
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

# Encode labels in column 'category'
df_train['ml_label']= le.fit_transform(df_train['category'])
df_val['ml_label']= le.fit_transform(df_val['category'])

#train['en_label'].unique()
#train['en_label'].value_counts()

#le.inverse_transform([2, 2, 1]

In [13]:
print(le.inverse_transform([0, 1]))

['hs' 'nhs']


In [14]:
# unique labels in the train set - only 2 labels
Label = pd.Series([0, 1])
y_train = np.array(Label.values.tolist())
y_val = df_val['category']


In [15]:
len(y_train)

2

In [16]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util
import torch

In [18]:
def SentTransf(model):   # returns the specifiec Sentence Transformers
  return SentenceTransformer(model)

In [19]:
# bert_model = 'l3cube-pune/gujarati-sentence-bert-nli'
bert_model = 'mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es'   # not giving good results
M1 = SentTransf(bert_model)

No sentence-transformers model found with name mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es. Creating a new one with MEAN pooling.


config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

C:\Users\User\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--mrm8488--distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/135 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [20]:
# bert_model = 'l3cube-pune/gujarati-sentence-bert-nli'
bert_model = 'hiiamsid/sentence_similarity_spanish_es'
M1 = SentTransf(bert_model)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

C:\Users\User\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--hiiamsid--sentence_similarity_spanish_es. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.51k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/242k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/480k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
# bert_model = 'l3cube-pune/gujarati-sentence-bert-nli'
bert_model = 'bertin-project/bertin-roberta-base-spanish'
M2 = SentTransf(bert_model)

No sentence-transformers model found with name bertin-project/bertin-roberta-base-spanish. Creating a new one with MEAN pooling.


config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

C:\Users\User\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\User\.cache\huggingface\hub\models--bertin-project--bertin-roberta-base-spanish. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at bertin-project/bertin-roberta-base-spanish and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/851k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/509k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.21M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [22]:
def get_embeddings_sent(text, model):
  #print(text)
  embds = model.encode(text)
  return embds

def get_embeddings(text, model):
  #print('text : ', text, 'len :', len(text))
  temp =  np.zeros(768)
  for line in text:   #sent_tokenize(text):
    #print('line : ', line)
    if len(line)>0:
      temp = np.add(temp, get_embeddings_sent(line, model))
  temp = np.divide(temp, len(text))
  #print(temp)
  return temp

In [23]:
#get_embeddings(tr_text['clean_text'][0])
#df_upsampled['bert_vec'] = df_upsampled['clean_text'].apply(lambda x: get_embeddings(x, M1))
df_train['indic_sim_vec'] = df_train['clean_text'].apply(lambda x: get_embeddings(x, M2))
df_train['xlm_vec'] = df_train['clean_text'].apply(lambda x: get_embeddings(x, M1))

In [24]:
#df_val['bert_vec'] = df_val['clean_text'].apply(lambda x: get_embeddings(x, M1))
df_val['indic_sim_vec'] = df_val['clean_text'].apply(lambda x: get_embeddings(x, M2))
df_val['xlm_vec'] = df_val['clean_text'].apply(lambda x: get_embeddings(x, M1))

In [ ]:
# df_test['xlm_vec'] = df_test['clean_text'].apply(lambda x: get_embeddings(x, M1))
# df_test['indic_sim_vec'] = df_test['clean_text'].apply(lambda x: get_embeddings(x, M2))

In [25]:
df_train['category'].unique()

array(['nhs', 'hs'], dtype=object)

In [26]:
Xtrain1 = df_train['xlm_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
Xtrain1 = np.array(Xtrain1.values.tolist())
Xtrain2 = df_train['indic_sim_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
Xtrain2 = np.array(Xtrain2.values.tolist())

In [ ]:
# Xtrain3 = df_train['distil_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
# Xtrain3 = np.array(Xtrain3.values.tolist())

In [27]:
y_train = df_train['ml_label']
y_train = np.array(y_train.values.tolist())


In [28]:
Xtest1 = df_val['xlm_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
Xtest1 = np.array(Xtest1.values.tolist())
Xtest2 = df_val['indic_sim_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
Xtest2 = np.array(Xtest2.values.tolist())

In [ ]:
# Xtst1 = df_test['xlm_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
# Xtst1 = np.array(Xtst1.values.tolist())
# Xtst2 = df_test['indic_sim_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
# Xtst2 = np.array(Xtst2.values.tolist())

In [ ]:
# Xtest3 = df_val['distil_vec']  #.to_numpy()  #.apply(pd.Series).reset_index()m4
# Xtest3 = np.array(Xtest3.values.tolist())

In [29]:
y_val = df_val['ml_label']
y_val = np.array(y_val.values.tolist())

In [30]:
from sklearn.utils import all_estimators

#estimators = all_estimators()

#for name, class_ in estimators:
#    if hasattr(class_, 'predict_proba'):
#        print(name)

estimators = all_estimators(type_filter='classifier')
for name, class_ in estimators:
    module_name = str(class_).split("'")[1].split(".")[1]
    class_name = class_.__name__
    print(f'from sklearn.{module_name} import {class_name}')

from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import CategoricalNB
from sklearn.multioutput import ClassifierChain
from sklearn.naive_bayes import ComplementNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
fr

In [31]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

clf5 = LogisticRegression(random_state=1)   # have to increase the number of repeatitions
clf1 = SVC(probability=True)  #DecisionTreeClassifier()   #GaussianNB()  # BernoulliNB()
clf4 = LinearSVC(penalty='l2', C=1.0)
clf2 = RandomForestClassifier(max_depth=2, random_state=0)

In [32]:
len(Xtest1), len(Xtest2), len(Xtrain1), len(Xtrain2)

(200, 200, 1400, 1400)

In [33]:
clf1 = clf1.fit(Xtrain1, y_train)
y_pred1 = clf1.predict(Xtest1)
y_pp1 = clf1.predict_proba(Xtest1)

In [34]:
y_pp1

array([[0.32392178, 0.67607822],
       [0.43189923, 0.56810077],
       [0.02573359, 0.97426641],
       [0.33158545, 0.66841455],
       [0.06363796, 0.93636204],
       [0.06130607, 0.93869393],
       [0.68768492, 0.31231508],
       [0.27564755, 0.72435245],
       [0.20723931, 0.79276069],
       [0.14345083, 0.85654917],
       [0.97452896, 0.02547104],
       [0.61958084, 0.38041916],
       [0.86446409, 0.13553591],
       [0.66453422, 0.33546578],
       [0.1250301 , 0.8749699 ],
       [0.11884695, 0.88115305],
       [0.22152842, 0.77847158],
       [0.78376503, 0.21623497],
       [0.63719347, 0.36280653],
       [0.04832228, 0.95167772],
       [0.09704796, 0.90295204],
       [0.26167755, 0.73832245],
       [0.46486203, 0.53513797],
       [0.5       , 0.5       ],
       [0.92084812, 0.07915188],
       [0.39386232, 0.60613768],
       [0.09089307, 0.90910693],
       [0.14893032, 0.85106968],
       [0.09560655, 0.90439345],
       [0.22362474, 0.77637526],
       [0.

In [35]:
y_pp1_max_ind = y_pp1.argmax(axis=1)   # to get the index of the maximum value
y_pp1_max_p = np.amax(y_pp1, axis=1)   # to get the maximum probability value

In [36]:
clf2 = clf2.fit(Xtrain2, y_train)
y_pred2 = clf2.predict(Xtest2)
y_pp2 = clf2.predict_proba(Xtest2)

In [37]:
y_pp2

array([[0.46973598, 0.53026402],
       [0.53677794, 0.46322206],
       [0.37581406, 0.62418594],
       [0.43977356, 0.56022644],
       [0.38117172, 0.61882828],
       [0.42875888, 0.57124112],
       [0.44504757, 0.55495243],
       [0.45195784, 0.54804216],
       [0.44286195, 0.55713805],
       [0.50273669, 0.49726331],
       [0.50675656, 0.49324344],
       [0.46241958, 0.53758042],
       [0.46080995, 0.53919005],
       [0.44743685, 0.55256315],
       [0.45282974, 0.54717026],
       [0.47936223, 0.52063777],
       [0.38847521, 0.61152479],
       [0.50753056, 0.49246944],
       [0.51721332, 0.48278668],
       [0.51575708, 0.48424292],
       [0.3473626 , 0.6526374 ],
       [0.46713915, 0.53286085],
       [0.47802206, 0.52197794],
       [0.44960214, 0.55039786],
       [0.49983491, 0.50016509],
       [0.38429888, 0.61570112],
       [0.44940693, 0.55059307],
       [0.44918378, 0.55081622],
       [0.40560177, 0.59439823],
       [0.4018253 , 0.5981747 ],
       [0.

In [38]:
y_pp2_max_ind = y_pp2.argmax(axis=1)   # to get the index of the maximum value
y_pp2_max_p = np.amax(y_pp2, axis=1)   # to get the maximum probability value

In [39]:
def get_pred(y_pred1, y_pp1_max_p, y_pred2, y_pp2_max_p):
  pred = []
  for i in range(len(y_pred1)):
    if y_pred1[i] == y_pred2[i]:
      pred.append(y_pred1[i])
    else:
      if y_pp1_max_p[i] >= y_pp2_max_p[i]:
        pred.append(y_pred1[i])
      else:
        pred.append(y_pred2[i])
  return pred

In [40]:
pred = get_pred(y_pred1, y_pp1_max_p, y_pred2, y_pp2_max_p)
pred_labels = list(le.inverse_transform(pred))
print(classification_report(df_val['category'], pred_labels))  #, target_names=target_names)

              precision    recall  f1-score   support

          hs       0.92      0.66      0.77       100
         nhs       0.73      0.94      0.82       100

    accuracy                           0.80       200
   macro avg       0.83      0.80      0.80       200
weighted avg       0.83      0.80      0.80       200



In [ ]:
y_pred11 = clf1.predict(Xtst1)
y_pred22 = clf2.predict(Xtst2)

In [ ]:
y_pp11 = clf1.predict_proba(Xtst1)
y_pp1_max_p1 = np.amax(y_pp11, axis=1)
y_pp22 = clf2.predict_proba(Xtst2)
y_pp1_max_p2 = np.amax(y_pp22, axis=1)

In [ ]:
pred = get_pred(y_pred11, y_pp1_max_p1, y_pred22, y_pp1_max_p2)
pred_labels = list(le.inverse_transform(pred))

In [ ]:
len(pred_labels),Xtst1.shape

(1032, (1032, 768))

In [ ]:
len(df_test)

1032

In [ ]:
#Submission
# to create .csv file consisting of tweet ids and predicted labels
y_pred_df = pd.DataFrame(data=pred_labels, columns=['final_prediction'])
Submisssion_Engm = pd.DataFrame()
Submisssion_Engm['id'] = df_test['id']
Submisssion_Engm['category'] = y_pred_df
Submisssion_Engm.to_csv('ENLISHnOW_FINAL_PROBN.csv',index = None)

In [ ]:
len(Submisssion_Engm)

1032

==================================================================================

In [ ]:
y_val

In [ ]:
a = y_pp2[0]
indices = np.where(a == a.max())

((array([1]),), 0.908879013554598)

In [ ]:
eclf = eclf.fit(mean_embedding1+mean_embedding2, y_train)
y_pred1 = eclf.predict(Xtest1+Xtest2)
y_pred1 = list(le.inverse_transform(y_pred1))
print(classification_report(y_val, y_pred1))

In [ ]:
 clf1.classes_

array([0, 1, 2])

In [ ]:
a = y_pp1[0]
indices = np.where(a == a.max())

In [ ]:
# to get the mean embeddigns of the classes in the training set
mean_embedding1 = df_upsampled.groupby('category')['distil_vec'].mean()
#mean_embedding1 = mean_embedding1.to_numpy()
mean_embedding1 = np.array(mean_embedding1.values.tolist())

In [ ]:
mean_embedding2 = df_upsampled.groupby('category')['indic_sim_vec'].mean()
#mean_embedding2 = mean_embedding2.to_numpy()
mean_embedding2 = np.array(mean_embedding2.values.tolist())

In [ ]:
print(df_upsampled.groupby('category')['distil_vec'].mean())

category
Homophobia                [0.015926234018458822, -0.015005965506236705, ...
Non-anti-LGBT+ content    [0.02178252166873464, -0.012524170068840415, -...
Transphobia               [0.03900217431390381, -0.039684160573744506, -...
Name: distil_vec, dtype: object


In [ ]:
eclf = VotingClassifier(estimators=[('LR', clf5), ('LSVC', clf4), ('SVC', clf1), ('RF', clf2)], voting='hard')
eclf = eclf.fit(mean_embedding1+mean_embedding2, y_train)
y_pred1 = eclf.predict(Xtest1+Xtest2)
y_pred1 = list(le.inverse_transform(y_pred1))
print(classification_report(y_val, y_pred1))  #, target_names=target_names)

                        precision    recall  f1-score   support

            Homophobia       0.20      0.69      0.31        42
Non-anti-LGBT+ content       0.98      0.81      0.89       748
           Transphobia       0.05      0.50      0.08         2

              accuracy                           0.81       792
             macro avg       0.41      0.67      0.43       792
          weighted avg       0.93      0.81      0.85       792



In [ ]:
 precision    recall  f1-score   support

            Homophobia       0.15      0.60      0.24        42
Non-anti-LGBT+ content       0.97      0.78      0.86       748
           Transphobia       0.04      0.50      0.07         2

              accuracy                           0.77       792
             macro avg       0.39      0.62      0.39       792
          weighted avg       0.93      0.77      0.83       792


In [ ]:
clf1 = LogisticRegression(random_state=1)
clf2 = BernoulliNB()
clf3 = LinearSVC(penalty='l2',C=1.0)
clf4 = RandomForestClassifier(max_depth=2, random_state=0)


eclf1 = VotingClassifier(estimators=[('LR', clf1), ('BNB', clf2)], voting='hard')
eclf1 = eclf1.fit(mean_embedding1, y_train)
y_pred1 = eclf1.predict(Xtest1)

eclf2 = VotingClassifier(estimators=[('SVC', clf3), ('RF', clf4)], voting='hard')
eclf2 = eclf2.fit(mean_embedding2, y_train)
y_pred2 = eclf2.predict(Xtest2)

eclf1 = VotingClassifier(estimators=[('VC1', eclf1), ('VC2', eclf2)], voting = 'hard')


y_pred1 = list(le.inverse_transform(y_pred1))
print(classification_report(y_val, y_pred1))  #, target_names=target_names)

In [ ]:
import numpy as np
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

from mvlearn.semi_supervised import CTClassifier
from mvlearn.datasets import load_UCImultifeature

In [ ]:
# Multi-view co-training semi-supervised learning
# Train a CTClassifier on all the labeled and unlabeled training data

label = pd.Series([0, 1])
y_train = np.array(label.values.tolist())

estimator1 =  GaussianNB()
estimator2 = RandomForestClassifier()
ctc = CTClassifier(estimator1, estimator2, random_state=1)
ctc.fit([mean1, mean2], y_train)


AttributeError: ignored

AttributeError: ignored

AttributeError: ignored

In [ ]:
df_train['bert_vec']=df_train['bert_vec'].fillna(0)
df_train['indic_sim_vec']=df_train['indic_sim_vec'].fillna(0)
df_val['bert_vec']=df_val['bert_vec'].fillna(0)
df_val['indic_sim_vec']=df_val['indic_sim_vec'].fillna(0)
df_test['bert_vec']=df_test['bert_vec'].fillna(0)
df_test['indic_sim_vec']=df_test['indic_sim_vec'].fillna(0)

In [ ]:
df_train['bert_vec'].shape, df_train['class'].shape

((3164,), (3164,))

In [ ]:
# # Use only the first 2 views as an example
# View0, View1 = df_train['bert_vec'], df_train['indic_sim_vec']

In [ ]:
# Import label encoder
from sklearn import preprocessing

le = preprocessing.LabelEncoder()

# Encode labels in column 'species'.
df_train['class']= le.fit_transform(df_train['Category'])
df_val['class']= le.transform(df_val['category'])
df_train['class'].unique()

#le.inverse_transform([2, 2, 1]

array([1, 0, 2])

In [ ]:
df_val['class'].unique()

array([0, 1, 2])

In [ ]:
# unique labels in the train set - only 2 labels
Label = pd.Series([1, 0, 2])
y_train = np.array(Label.values.tolist())

In [ ]:
Label = pd.Series([1, 0, 2])
y_val = np.array(Label.values.tolist())

In [ ]:
len(y_train)

3

In [ ]:
y_pred_ct = ctc.predict([Xtest1, Xtest2])

In [ ]:
# # to get the mean embeddigns of the classes in the validation set
#mean_embedding1 = df_val.groupby('category') ['bert_vec'].mean()

#mean_embedding1 = mean_embedding1.to_numpy()
# mean_embedding1 = np.array(mean_embedding1.values.tolist())

In [ ]:
#mean_embedding1 = np.array(pd.DataFrame(mean_embedding1).fillna(0))

In [ ]:
# train1=np.array(pd.DataFrame(train1).fillna(0))

In [ ]:
# train1

In [ ]:
# train2.shape,train1.shape

((3164, 768), (3164, 768))

In [ ]:
# train2=df_train['indic_sim_vec'].values.tolist()

In [ ]:
# train2=pd.DataFrame(train2).fillna(0)

In [ ]:
# train2=df_train['indic_sim_vec'].to_numpy()

In [ ]:
#mean_embedding2=np.array(pd.DataFrame(mean_embedding2).fillna(0))

In [ ]:
mean1 = mean_embedding1[:2]
mean2 = mean_embedding2[:2]

In [ ]:
# unique labels in the train set - only 2 labels
label = pd.Series([0, 1,2])
y_train = np.array(label.values.tolist())
y_val = df_val['category']

In [ ]:
!pip install mvlearn